In [9]:
import pandas as pd
import os

os.chdir("/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets_Codigo")

df_clied = pd.read_csv('Mer_clied.txt', encoding='latin1')
df_clifd = pd.read_csv('Mer_Clifd.txt', encoding="latin1")
df_clird = pd.read_csv('Mer_Clird.txt', encoding='latin1')
df_clifad = pd.read_csv('Mer_Clifad.txt', encoding='latin1')
df_articulos = pd.read_csv('/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets/Articulos.txt', encoding='latin1')

df_merged = pd.concat([df_clied, df_clifd, df_clird, df_clifad])

df_merged['Fec'] = df_merged['Fecncre'].fillna(df_merged['Fecfac']).fillna(df_merged['Fecrem'])
df_merged.drop(['Fecncre', 'Fecfac', 'Fecrem'], axis=1, inplace=True)

df_merged['Fec'] = pd.to_datetime(df_merged['Fec'], format="%d/%m/%y")

df_merged['Año'] = df_merged['Fec'].dt.year
df_merged['Mes'] = df_merged['Fec'].dt.month

df_resultado = df_merged.groupby(['Claart', 'Año', 'Mes','Numfac']).agg({'Cant': 'sum', 'Fec': 'nunique'}).reset_index()
df_resultado

,Claart,Año,Mes,Numfac,Cant,Fec
0,0118592,2023,5,ZI086291,1.0,1
1,01ACC,2017,11,ZI010505,15.0,1
2,01ACC,2017,12,ZI011195,60.0,1
3,01ACC,2018,2,ZI011765,200.0,1
4,01ACC,2018,2,ZI011996,2.0,1
...,...,...,...,...,...,...
210557,Venta,2023,11,ZI098489,27.0,1
210558,Venta,2023,11,ZI098592,30.0,1
210559,Venta,2023,11,ZI098671,13.0,1
210560,Venta,2023,11,ZI098732,21.0,1


In [10]:
import pandas as pd
import os
df_clind = pd.read_csv('Mer_Clind.txt', encoding='latin1')
df_merged = pd.merge(df_resultado, df_clind[['Numfac']], on='Numfac', how='left', indicator=True)
df_merged = df_merged[df_merged['_merge'] == 'left_only']
df_merged.drop(columns=['_merge'], inplace=True)
df_merged = df_merged.copy()
df_merged.drop(columns=['Numfac'], inplace=True)
df_merged.drop(['Fec'], axis=1, inplace=True)
df_merged

,Claart,Año,Mes,Cant
0,0118592,2023,5,1.0
1,01ACC,2017,11,15.0
3,01ACC,2018,2,200.0
4,01ACC,2018,2,2.0
5,01ACC,2018,10,60.0
...,...,...,...,...
217436,Venta,2023,11,27.0
217437,Venta,2023,11,30.0
217438,Venta,2023,11,13.0
217439,Venta,2023,11,21.0


In [11]:
import pandas as pd
os.chdir("/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets_Codigo")
df_articulos = pd.read_csv('Articulos.txt', encoding='latin1')
clave_articulo = input("Por favor, ingrese la clave del artículo que desea filtrar: ")
df_filtrado = df_merged[(df_merged['Claart'] == clave_articulo) & (df_merged['Año'] >= 2017)]
df_filtrado['Mes'] = df_filtrado['Mes'].astype(int)
df_ventas_por_mes = df_filtrado.groupby(['Claart', 'Año', 'Mes'])['Cant'].sum().reset_index()
df_ventas = pd.merge(df_ventas_por_mes, df_articulos[['Claart', 'Desart']], on='Claart', how='left')
print("Ventas por mes y año para la clave del artículo:", clave_articulo)
df_ventas = df_ventas[['Claart', 'Desart', 'Cant', 'Mes', 'Año']]
df_ventas

Ventas por mes y año para la clave del artículo: 0118592


/var/folders/cn/s8xbm9k519n2w4bqvq70nvmr0000gn/T/ipykernel_23354/4244048313.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['Mes'] = df_filtrado['Mes'].astype(int)


,Claart,Desart,Cant,Mes,Año
0,0118592,Tornillo de banco truper 18592,1.0,5,2023


In [ ]:
df_ventas.to_csv('/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets_Codigo/Mer_Filtrado.txt', index=False, encoding="latin1")